- pipeline
- tokenizer
- model
- datasets
- evaluate
- trainer

导入相关包
加载数据集
数据集划分
数据集预处理
创建模型
设置评估函数
配置训练参数
创建训练参数
模型训练、评估、预测
模型预测

显存占用分析
- 模型权重  4Bytes*模型参数量
- 优化器状态 8Bytes*模型参数量
- 梯度  4Bytes*模型参数量
- 前向激活值 取决于序列长度、隐层维度、batch大小等多个因素

显存占用优化策略分析
- gradient accmulation 梯度累加
- gradient checkpoints 正向传播的过程中边丢弃一些中间变量，在反向传播的时候再重新计算
- adafactor optimizer 不计算梯度的二阶正定矩阵，用了另外一种相似矩阵分解技术，加快了模型显存的占用
- freeze model 其实这是一种策略，而不是一种方法
- data length 输入数据的长度

| 优化策略                                      | 优化对象         | 显存占用 | 训练时间 |
|---------------------------------------------|-----------------|----------|----------|
| Baseline (BS 32, MaxLength 128)             | -               | 15.2G    | 64s      |
| + Gradient Accumulation (BS 1, GA 32)       | 前向激活值      |   7.4G       |    259s      |
| + Gradient Checkpoints (BS 1, GA 32)        | 前向激活值      |    7.2G      |     422s     |
| + Adafactor Optimizer (BS 1, GA 32)         | 优化器状态      |     5.0G     |     406s     |
| + Freeze Model (BS 1, GA 32)                | 前向激活值 / 梯度 |      3.4G    |      180s    |
| + Data Length (BS 1, GA 32, MaxLength 64)   | 前向激活值      |     3.3G     |      126s    |

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import evaluate
from transformers import DataCollatorWithPadding
from transformers import pipeline

# 定义数据预处理函数，用于将文本转换为模型可以接受的格式，并添加标签
def process_function(examples):
    # 使用分词器对评论进行编码，并设置最大长度、截断和填充
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    # 将原始数据集中的标签添加到编码后的字典中
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

# 定义评估指标计算函数，用于在验证或测试时计算模型性能
def eval_metric(eval_predict):
    predictions, labels = eval_predict  # 分离预测结果和真实标签
    predictions = predictions.argmax(axis=-1)  # 获取每个样本的最大概率对应的类别作为预测类别
    acc = acc_metric.compute(predictions=predictions, references=labels)  # 计算准确率
    f1 = f1_metirc.compute(predictions=predictions, references=labels)  # 计算F1分数
    acc.update(f1)  # 将F1分数更新到准确率字典中，以便一起返回
    return acc

# 加载CSV文件格式的数据集，并指定分割为训练集
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")

# 过滤掉评论为空的数据项
dataset = dataset.filter(lambda x: x["review"] is not None)

# 将数据集划分为训练集和测试集，测试集占10%
datasets = dataset.train_test_split(test_size=0.1)

# 加载预训练的分词器，这里使用的是中文的macBERT-large模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

# 对数据集进行批量处理，应用process_function函数来准备模型输入，并移除原始列名
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)

# 加载预训练的序列分类模型，这里同样选择的是中文的macBERT-large模型
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")

# 加载自定义的准确率和F1评分度量脚本（假设这些脚本已经存在）
acc_metric = evaluate.load("./metric_accuracy.py")
f1_metirc = evaluate.load("./metric_f1.py")

# 配置训练参数
train_args = TrainingArguments(
    output_dir="./checkpoints",      # 输出目录，保存检查点等文件
    per_device_train_batch_size=2,   # 每个设备上的训练批次大小
    gradient_accumulation_steps=32,  # 梯度累积步数，用于模拟更大的批次大小
    gradient_checkpointing=True,     # 启用梯度检查点技术以节省显存
    optim="adafactor",               # 使用Adafactor优化算法
    per_device_eval_batch_size=4,    # 每个设备上的评估批次大小
    num_train_epochs=1,              # 总共训练的轮数
    logging_steps=10,                # 每隔多少步打印一次日志信息
    eval_strategy="epoch",           # 在每个epoch结束时进行评估
    save_strategy="epoch",           # 在每个epoch结束时保存模型
    save_total_limit=3,              # 最多保存3个检查点
    learning_rate=2e-5,              # 初始学习率
    weight_decay=0.001,              # 权重衰减系数
    metric_for_best_model="f1",      # 根据哪个指标选择最佳模型
    load_best_model_at_end=True      # 在训练结束后加载最优模型
)

# 冻结BERT层的参数，只训练分类头部分。这有助于减少训练时间和资源消耗。
for name, param in model.bert.named_parameters():
    param.requires_grad = False

# 创建Trainer实例，负责管理模型训练、评估和预测过程
trainer = Trainer(
    model=model, 
    args=train_args, 
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],  # 训练数据集
    eval_dataset=tokenized_datasets["test"],    # 测试数据集
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # 数据整理器，确保batch内的序列长度一致
    compute_metrics=eval_metric  # 自定义的评估指标计算函数
)

# 开始训练模型
trainer.train()

# 在测试集上评估模型性能
trainer.evaluate(tokenized_datasets["test"])

# 在测试集上进行预测并输出预测结果
trainer.predict(tokenized_datasets["test"])

In [ ]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)